In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
import nltk

from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score, recall_score, precision_score

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks/A_SNLP')

# 1. Load the dataset (5 points)

In [48]:
df =pd.read_csv('blogtext.csv')
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [0]:
df = df[0:10000]

In [50]:
df.shape

(10000, 7)

In [51]:
df.columns.values

array(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'],
      dtype=object)

In [52]:
print(df['text'][5])

             I had an interesting conversation with my Dad this morning.  We were talking about where Koreans put their money.  Invariably, they have a lot of real estate and cash.  (Cash would include short term investments under one year as well as savings accounts.)  The reason?  Real estate makes money here.  A lot of money.  I've seen surveys of Seoul real estate rising about 10-15% PER YEAR for long stretches, even after taking into account the 1997 Crisis (referred to as the IMF crisis here, although it was the IMF that bailed Korea out).  Compare that to Korean corporate bonds which fell 90-99% in 1997 and only modestly recovered, and a local stock market (represented by KOSPI, or their version of the Dow Jones Index) that has not gone appreciably above its 1980s high of 1,000 points (it is now about 800 points, see  urlLink link ) and you can see why real estate makes sense here.  But back to the conversation...I noted that here a 'real big' or 'elite' real estate investor has

In [53]:
print(df['text'])

0                  Info has been found (+/- 100 pages,...
1                  These are the team members:   Drewe...
2                  In het kader van kernfusie op aarde...
3                        testing!!!  testing!!!          
4                    Thanks to Yahoo!'s Toolbar I can ...
                              ...                        
9995            take me home with you forever where I ...
9996            seductive secretness behind doors warn...
9997            For being so kind to me when I need yo...
9998            blurry outside sounds as people mingle...
9999            my body feels broken while my mind rej...
Name: text, Length: 10000, dtype: object


# Preprocess rows of the “text” column (7.5 points)

In [0]:
from bs4 import BeautifulSoup
import re

In [55]:
len(df['text'])

10000

In [56]:
df['text'].loc[1]

'           These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail          '

In [57]:
df['text'].loc[2]

"           In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, like gold,' as it was missing. If anyone has the full text, please distribute. I am not responsible for the accuracy of this information. Converted to HTML by Dionisio@InfiNet.com 11/13/98. (Did a little spell-checking and some minor edits too.) Stolen from  urlLink http://my.ohio.voyager.net/~dionisio/fun/m...own-h-bomb.html  and reformatted the HTML. It now validates to XHTML 1.0 Strict. How to Build an H-Bomb Making and owning an H-bomb is the kind of challenge real Americans seek. Who wants to 

1.   Remove unwanted characters
2.   Convert text to lowercase
3.   Remove unwanted spaces
4.   Remove stopwords

In [58]:
# Doing Pre processing. 

for i in range(df.shape[0]):
  example1 = BeautifulSoup(df['text'].loc[i])               # Beautifying the text  
  df['text'].loc[i] = example1.get_text()
  df['text'][i] = re.sub("[^a-zA-Z]", " ", df['text'][i])   # Removing unwanted characters
  df['text'][i] = df['text'][i].lower()                     # Lowercase characters
  df['text'][i] =  df['text'][i].strip() 
  df['text'][i] = re.sub("\s\s+", " ", df['text'][i])       # Removing Space

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

In [59]:
# Comparing with old values
df['text'].loc[1]

'these are the team members drewes van der laag urllink mail ruiyu xie urllink mail bryan aaldering me urllink mail'

In [60]:
df['text'].loc[2]

'in het kader van kernfusie op aarde maak je eigen waterstofbom how to build an h bomb from ascott tartarus uwa edu au andrew scott newsgroups rec humor subject how to build an h bomb humorous date feb gmt organization the university of western australia original file dated th november seemed to be a transcript of a seven days article poorly formatted and corrupted i have added the text between examine under a microscope and malleable like gold as it was missing if anyone has the full text please distribute i am not responsible for the accuracy of this information converted to html by dionisio infinet com did a little spell checking and some minor edits too stolen from urllink http my ohio voyager net dionisio fun m own h bomb html and reformatted the html it now validates to xhtml strict how to build an h bomb making and owning an h bomb is the kind of challenge real americans seek who wants to be a passive victim of nuclear war when with a little effort you can be an active participa

In [61]:
# Comparing with old values
print(df['text'])

0       info has been found pages and mb of pdf files ...
1       these are the team members drewes van der laag...
2       in het kader van kernfusie op aarde maak je ei...
3                                         testing testing
4       thanks to yahoo s toolbar i can now capture th...
                              ...                        
9995    take me home with you forever where i may rest...
9996    seductive secretness behind doors warning me t...
9997    for being so kind to me when i need you most f...
9998    blurry outside sounds as people mingle and pas...
9999    my body feels broken while my mind rejoices at...
Name: text, Length: 10000, dtype: object


In [62]:
# Removing stop words
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
# Splitting the sentence into words to remove stopwords
df['text'] = df['text'].apply( lambda t : " ".join(word for word in t.split() if word not in stop_words)) 

In [65]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages mb pdf files wait untill team...
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...


# 3. As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence (7.5 points)

In [66]:
data_blog = pd.DataFrame(columns = ["text", "labels"])
df['age'] = df['age'].astype(str)
data_blog["labels"] = df[['gender','age','topic','sign']].apply(lambda x: ','.join(x), axis = 1) 
data_blog["text"] = df["text"]
data_blog.head()

,text,labels
0,info found pages mb pdf files wait untill team...,"male,15,Student,Leo"
1,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"


# 4. Separate features and labels, and split the data into training and testing (5 points)

In [0]:
features = data_blog["text"]
labels = data_blog["labels"].str.lower()
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size = 0.3, random_state = 7)

# 5. Vectorize the features (5 points)

In [68]:
# Creating Bag of words
vectorizer = CountVectorizer(min_df = 2,ngram_range = (1,2),stop_words = "english")
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print("X_train shape & sample",X_train.shape)


X_train shape & sample (7000, 59134)


In [69]:
print(vectorizer)
print(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=2,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
  (0, 48752)	2
  (0, 48879)	1
  (0, 48683)	2
  (0, 23652)	1
  (0, 8866)	1
  (0, 21128)	1
  (0, 51449)	1
  (0, 5991)	1
  (0, 55805)	7
  (0, 41043)	2
  (0, 8902)	1
  (0, 11131)	1
  (0, 44183)	1
  (0, 49417)	1
  (0, 3202)	1
  (0, 38048)	1
  (0, 27044)	1
  (0, 10486)	1
  (0, 22026)	1
  (0, 43859)	1
  (0, 16556)	1
  (0, 17663)	1
  (0, 32004)	2
  (0, 54882)	2
  (0, 41628)	1
  :	:
  (6999, 10136)	1
  (6999, 42613)	2
  (6999, 20944)	1
  (6999, 20590)	1
  (6999, 36398)	1
  (6999, 45311)	1
  (6999, 23117)	1
  (6999, 30477)	1
  (6999, 2243)	1
  (6999, 2785)	1
  (6999, 20592)	1
  (6999

# Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. Check below image for reference (5 points)

In [70]:
vectorizer_labels = CountVectorizer(min_df = 1,ngram_range = (1,1),stop_words = "english")
labels_vector = vectorizer_labels.fit_transform(labels)
vectorizer_labels.vocabulary_

{'13': 0,
 '14': 1,
 '15': 2,
 '16': 3,
 '17': 4,
 '23': 5,
 '24': 6,
 '25': 7,
 '26': 8,
 '27': 9,
 '33': 10,
 '34': 11,
 '35': 12,
 '36': 13,
 '37': 14,
 '38': 15,
 '39': 16,
 '40': 17,
 '41': 18,
 '42': 19,
 '43': 20,
 '44': 21,
 '45': 22,
 '46': 23,
 'accounting': 24,
 'aquarius': 25,
 'aries': 26,
 'arts': 27,
 'automotive': 28,
 'banking': 29,
 'businessservices': 30,
 'cancer': 31,
 'capricorn': 32,
 'communications': 33,
 'consulting': 34,
 'education': 35,
 'engineering': 36,
 'fashion': 37,
 'female': 38,
 'gemini': 39,
 'humanresources': 40,
 'indunk': 41,
 'internet': 42,
 'investmentbanking': 43,
 'law': 44,
 'lawenforcement': 45,
 'leo': 46,
 'libra': 47,
 'libraries': 48,
 'male': 49,
 'marketing': 50,
 'media': 51,
 'museums': 52,
 'non': 53,
 'pisces': 54,
 'profit': 55,
 'publishing': 56,
 'recreation': 57,
 'religion': 58,
 'sagittarius': 59,
 'science': 60,
 'scorpio': 61,
 'security': 62,
 'sports': 63,
 'student': 64,
 'taurus': 65,
 'technology': 66,
 'telecommun

In [71]:
labeled_classes = []  
for key in vectorizer_labels.vocabulary_.keys():
    labeled_classes.append(key)
    
print(sorted(labeled_classes))

['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', 'accounting', 'aquarius', 'aries', 'arts', 'automotive', 'banking', 'businessservices', 'cancer', 'capricorn', 'communications', 'consulting', 'education', 'engineering', 'fashion', 'female', 'gemini', 'humanresources', 'indunk', 'internet', 'investmentbanking', 'law', 'lawenforcement', 'leo', 'libra', 'libraries', 'male', 'marketing', 'media', 'museums', 'non', 'pisces', 'profit', 'publishing', 'recreation', 'religion', 'sagittarius', 'science', 'scorpio', 'security', 'sports', 'student', 'taurus', 'technology', 'telecommunications', 'virgo']


# 7. Transform the labels

In [0]:
mlb = MultiLabelBinarizer(classes = labeled_classes)  # initialising multilabelbinariser with all unique possible classes

In [73]:
# Converting entire se of labels into format required by mlb
labels = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in labels]]
labels[30]

['male', '33', 'investmentbanking', 'aquarius']

In [74]:
labels_trans = mlb.fit(labels) # transforming entire set of lables
labels_trans

MultiLabelBinarizer(classes=['male', '15', 'student', 'leo', '33',
                             'investmentbanking', 'aquarius', 'female', '14',
                             'indunk', 'aries', '25', 'capricorn', '17',
                             'gemini', '23', 'non', 'profit', 'cancer',
                             'banking', '37', 'sagittarius', '26', '24',
                             'scorpio', '27', 'education', '45', 'engineering',
                             'libra', ...],
                    sparse_output=False)

In [0]:
#Convert Y_train into a format as required by mlb 
Y_train = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y_train]]

In [76]:
print(labels[25])
print(Y_train[25])

['male', '33', 'investmentbanking', 'aquarius']
['male', '16', 'indunk', 'cancer']


In [77]:
Y_train_trans = mlb.transform(Y_train)
Y_train_trans[25]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['communicationsmedia', 'lawenforcementsecurity', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [78]:
#Convert Y_test into a format as required by mlb 
Y_test = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y_test]]
Y_test_trans = mlb.transform(Y_test) # transforming test labels.
print(Y_test[25])

['male', '35', 'technology', 'aries']


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['communicationsmedia', 'lawenforcementsecurity', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


In [79]:
len(mlb.classes_)

69

In [80]:
mlb.classes_

array(['male', '15', 'student', 'leo', '33', 'investmentbanking',
       'aquarius', 'female', '14', 'indunk', 'aries', '25', 'capricorn',
       '17', 'gemini', '23', 'non', 'profit', 'cancer', 'banking', '37',
       'sagittarius', '26', '24', 'scorpio', '27', 'education', '45',
       'engineering', 'libra', 'science', '34', '41', 'communications',
       'media', 'businessservices', 'sports', 'recreation', 'virgo',
       'taurus', 'arts', 'pisces', '44', '16', 'internet', 'museums',
       'libraries', 'accounting', '39', '35', 'technology', '36', 'law',
       '46', 'consulting', 'automotive', '42', 'religion', '13',
       'fashion', '38', '43', 'publishing', '40', 'marketing',
       'lawenforcement', 'security', 'humanresources',
       'telecommunications'], dtype=object)

# 8. Choose a classifier - (5 points)

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [0]:
clf = LogisticRegression(solver='lbfgs',max_iter=1000)
clf = OneVsRestClassifier(clf)

# 9. Fit the classifier, make predictions and get the accuracy (5 points)

In [83]:
# Fitting training  data
clf.fit(X_train,Y_train_trans) 

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 16 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 17 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 33 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 36 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 37 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 45 is present in all training 

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [84]:
# Train Accuracy
print("Train Accuracy:",clf.score(X_train,Y_train_trans))

Train Accuracy: 0.92


In [0]:
Y_pred = clf.predict(X_test)

In [86]:
print("Accuracy on Test Data:" + str(accuracy_score(Y_test_trans, Y_pred)))
print("F1 Score_micro: " + str(f1_score(Y_test_trans, Y_pred, average='micro')))
print("F1_Score_macro: " + str(f1_score(Y_test_trans, Y_pred, average='macro')))
print("Precision_micro: " + str(precision_score(Y_test_trans, Y_pred, average='micro')))
print("Precision_macro: " + str(precision_score(Y_test_trans, Y_pred, average='macro')))
print("Recall_micro: " + str(recall_score(Y_test_trans, Y_pred, average='micro')))
print("Recall_macro: " + str(recall_score(Y_test_trans, Y_pred, average='macro')))

Accuracy on Test Data:0.32233333333333336
F1 Score_micro: 0.6481049562682215
F1_Score_macro: 0.2655935700850831
Precision_micro: 0.7685836118474128
Precision_macro: 0.43966260699366977
Recall_micro: 0.5602789212803495
Recall_macro: 0.20910949269438797


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Very low test accuracy achieved here

# 10.  Print true label and predicted label for any five examples (7.5 points)

In [0]:
Y_pred_inv = mlb.inverse_transform(Y_pred)   # inverse transforming predited label data
Y_test_trans_inv =  mlb.inverse_transform(Y_test_trans) # inverse transforming original test label data

In [88]:
print("Example 1 - predicted :",Y_pred_inv[0])
print("Example 1 - Actual :",Y_test_trans_inv[0])
print("Example 1 - Actual_before mlb transformation :",Y_test[0])

Example 1 - predicted : ('male', 'aries')
Example 1 - Actual : ('male', 'aries', '35', 'technology')
Example 1 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [89]:
print("Example 2 - predicted :",Y_pred_inv[25])
print("Example 2 - Actual :",Y_test_trans_inv[25])
print("Example 2 - Actual_before mlb transformation :",Y_test[25])

Example 2 - predicted : ('male', 'aries', '35', 'technology')
Example 2 - Actual : ('male', 'aries', '35', 'technology')
Example 2 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [90]:
print("Example 3 - predicted :",Y_pred_inv[150])
print("Example 3 - Actual :",Y_test_trans_inv[150])
print("Example 3 - Actual_before mlb transformation :",Y_test[150])

Example 3 - predicted : ('male', 'technology')
Example 3 - Actual : ('male', 'aries', '35', 'technology')
Example 3 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [91]:
print("Example 4 - predicted :",Y_pred_inv[250])
print("Example 4 - Actual :",Y_test_trans_inv[250])
print("Example 4 - Actual_before mlb transformation :",Y_test[250])

Example 4 - predicted : ('male', 'aries')
Example 4 - Actual : ('male', 'student', 'leo', '14')
Example 4 - Actual_before mlb transformation : ['male', '14', 'student', 'leo']


In [92]:
print("Example 5 - predicted :",Y_pred_inv[350])
print("Example 5 - Actual :",Y_test_trans_inv[350])
print("Example 5 - Actual_before mlb transformation :",Y_test[350])

Example 5 - predicted : ('female', 'indunk', '17', 'scorpio')
Example 5 - Actual : ('female', 'indunk', '17', 'scorpio')
Example 5 - Actual_before mlb transformation : ['female', '17', 'indunk', 'scorpio']
